Welcome to the torchkbnufft demo! Let's get started...

# Data Loading and Formatting
First, we need to load some data. For this we can use scipy.io to load a matlab file with complex k-space data.

In [1]:
import scipy.io as sio

obj = sio.loadmat('demo_data.mat')
kdata_np, ktraj_np, smap_np = obj['kdata'], obj['ktraj'], obj['smap']

Let's look at the size of this data set...

In [2]:
print('kdata size: {}'.format(kdata_np.shape))
print('ktraj size: {}'.format(ktraj_np.shape))
print('smap size: {}'.format(smap_np.shape))

kdata size: (8, 405, 512)
ktraj size: (2, 405, 512)
smap size: (8, 256, 256)


This is a radial data set, so we have 405 spokes, each of length 512. In this case we've already calculated the sensitivity maps, but in practical settings you'll need to calculate them on your own.

In order to use TorchKbNufft, we need to reshape this data so the k-space points are in a single vector. We also need to unsqueeze a batch dimension. Finally, we need to separate real and imaginary components.

In [3]:
import torch
import numpy as np

# this "stacks" real and imaginary components along the second dimension
print('stacking real/imaginary...')
kdata = torch.stack((torch.tensor(np.real(kdata_np)), torch.tensor(np.imag(kdata_np))), dim=1)
ktraj = torch.tensor(ktraj_np)
print('kdata shape: {}'.format(kdata.shape))
print('ktraj shape: {}'.format(ktraj.shape))

# flatten k-space coordinates...
print('flattening k-space')
kdata = kdata.view(kdata.shape[0], kdata.shape[1], -1)
ktraj = ktraj.view(ktraj.shape[0], -1)
print('kdata shape: {}'.format(kdata.shape))
print('ktraj shape: {}'.format(ktraj.shape))

# unsqueeze the batch dimension
print('unsqueezing batch...')
kdata = kdata.unsqueeze(0)
ktraj = ktraj.unsqueeze(0)
print('kdata shape: {}'.format(kdata.shape))
print('ktraj shape: {}'.format(ktraj.shape))

stacking real/imaginary...
kdata shape: torch.Size([8, 2, 405, 512])
ktraj shape: torch.Size([2, 405, 512])
flattening k-space
kdata shape: torch.Size([8, 2, 207360])
ktraj shape: torch.Size([2, 207360])
unsqueezing batch...
kdata shape: torch.Size([1, 8, 2, 207360])
ktraj shape: torch.Size([1, 2, 207360])


We also need to convert the sensitivity maps to a tensor...

In [4]:
smap = torch.stack((torch.tensor(np.real(smap_np)), torch.tensor(np.imag(smap_np))), dim=1)
smap = smap.unsqueeze(0)
print('smap shape: {}'.format(smap.shape))

smap shape: torch.Size([1, 8, 2, 256, 256])


# Adjoint SENSE NUFFTs

Now we're ready to use the package!

Using TorchKbNufft is essentially the same as using PyTorch. First, you define a layer (either a forward or adjoint NUFFT). Then, you can apply that layer.

Let's use the NUFFT and plot!

We can improve the image with some density compensation...

# Sparse Matrix Mode

The operation we just covered uses table interpolation. In many cases, using sparse matrices as a backend can be faster. We can easily use sparse matrices in TorchKbNufft with a simple precomputation step.

# Toeplitz forward/backward NUFFTs

Toeplitz NUFFTs are a very fast way to compute forward/backward nuffts without any interpolation steps. They only require precalculation of an FFT filter.

Now let's test it...

And the Toeplitz version...